In [1]:
import pandas as pd
import numpy as np
import os
from geopy.distance import geodesic
from ipywidgets import interactive
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
new_df_ContainerActivities = pd.read_csv('/Users/annele/Library/CloudStorage/OneDrive-zebrafant.ai/BA/Data/ContainerActivities_coordinates.csv') 

In [3]:
new_df_ContainerActivities

,TRANSACTION_ID,CONTAINER_ID,SLIDER_LEVEL,RECORDED_AT,COMMENT,IS_EMPTIED,LATITUDE,LONGITUDE,UPDATED_AT,LOCATION_ID,GEO_LAT,GEO_LON,LATITUDE_DIFF,LONGITUDE_DIFF,DISTANCE
0,1,1001101101100271111,30,2023-09-26 06:36:48.0,NaN,1,49.455948,7.791700,NaN,100110110110027,49.456130,7.791667,-0.000182,0.000033,20.382686
1,2,1001101101100271211,20,2023-09-26 06:36:05.0,NaN,1,49.455948,7.791700,NaN,100110110110027,49.456130,7.791667,-0.000182,0.000033,20.382686
2,3,1001101101100271311,10,2023-09-26 06:37:16.0,NaN,0,49.455948,7.791700,NaN,100110110110027,49.456130,7.791667,-0.000182,0.000033,20.382686
3,4,1001101101100191111,0,2023-09-26 06:51:15.0,NaN,0,49.448544,7.769812,NaN,100110110110019,49.448608,7.769743,-0.000064,0.000069,8.700529
4,5,1001101101100191211,10,2023-09-26 06:51:44.0,NaN,0,49.448544,7.769812,NaN,100110110110019,49.448608,7.769743,-0.000064,0.000069,8.700529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30524,57277,1001101102101271311,20,2024-09-06 11:53:47.0,NaN,0,49.475612,7.637055,2024-09-30 15:32:27.0,100110110210127,49.475595,7.636934,0.000017,0.000121,8.940148
30525,57287,1001101102101031111,70,2024-09-06 12:53:16.0,NaN,1,49.470207,7.642706,2024-09-30 15:32:27.0,100110110210103,49.470222,7.642667,-0.000015,0.000039,3.269312
30526,57288,1001101102101031211,30,2024-09-06 12:53:25.0,NaN,1,49.470207,7.642706,2024-09-30 15:32:27.0,100110110210103,49.470222,7.642667,-0.000015,0.000039,3.269312
30527,57289,1001101102101031311,20,2024-09-06 12:53:08.0,NaN,0,49.470207,7.642706,2024-09-30 15:32:27.0,100110110210103,49.470222,7.642667,-0.000015,0.000039,3.269312


# Classfication of the data

In [4]:
# drop rows on the same location and date
new_df_ContainerActivities['RECORDED_AT'] = pd.to_datetime(new_df_ContainerActivities['RECORDED_AT'])
new_df_ContainerActivities['RECORDED_DATE'] = new_df_ContainerActivities['RECORDED_AT'].dt.date
df_ContainerActivities_unique = new_df_ContainerActivities.drop_duplicates(subset=['LOCATION_ID', 'RECORDED_DATE'])
new_df_ContainerActivities = df_ContainerActivities_unique.copy()
new_df_ContainerActivities

,TRANSACTION_ID,CONTAINER_ID,SLIDER_LEVEL,RECORDED_AT,COMMENT,IS_EMPTIED,LATITUDE,LONGITUDE,UPDATED_AT,LOCATION_ID,GEO_LAT,GEO_LON,LATITUDE_DIFF,LONGITUDE_DIFF,DISTANCE,RECORDED_DATE
0,1,1001101101100271111,30,2023-09-26 06:36:48,NaN,1,49.455948,7.791700,NaN,100110110110027,49.456130,7.791667,-0.000182,0.000033,20.382686,2023-09-26
3,4,1001101101100191111,0,2023-09-26 06:51:15,NaN,0,49.448544,7.769812,NaN,100110110110019,49.448608,7.769743,-0.000064,0.000069,8.700529,2023-09-26
6,7,1001101101100161111,30,2023-09-26 06:57:22,NaN,1,49.449840,7.770489,NaN,100110110110016,49.450380,7.771540,-0.000540,-0.001051,97.029102,2023-09-26
9,10,1001101101100151111,20,2023-09-26 07:10:08,NaN,1,49.454563,7.772812,NaN,100110110110015,49.454544,7.772941,0.000019,-0.000129,9.588712,2023-09-26
12,13,1001101101100501111,20,2023-09-26 07:21:51,NaN,1,49.450213,7.764416,NaN,100110110110050,49.450027,7.763767,0.000186,0.000649,51.405201,2023-09-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30513,57257,1001101102101011111,50,2024-09-06 11:19:10,NaN,1,49.496529,7.552427,2024-09-30 15:32:27.0,100110110210101,49.496528,7.552500,0.000001,-0.000073,5.282977,2024-09-06
30516,57263,1001101102101291111,40,2024-09-06 11:34:43,NaN,1,49.483794,7.624123,2024-09-30 15:32:27.0,100110110210129,49.483722,7.624139,0.000072,-0.000016,8.072405,2024-09-06
30519,57269,1001101102101261111,60,2024-09-06 11:42:45,NaN,1,49.482377,7.631627,2024-09-30 15:32:27.0,100110110210126,49.482278,7.631583,0.000099,0.000044,11.445704,2024-09-06
30522,57275,1001101102101271111,50,2024-09-06 11:53:38,NaN,1,49.475612,7.637055,2024-09-30 15:32:27.0,100110110210127,49.475595,7.636934,0.000017,0.000121,8.940148,2024-09-06


In [5]:
# Parameter für die Klassifikationsregeln
relative_diff_threshold = 20  # Maximal erlaubte Differenz für Konsistenz
hard_outlier_threshold = 400  # Harte Grenze für Outliers
outlier_absolute_threshold = 150  # Schwelle für Abweichung vom Mittelwert
outlier_multiplier = 3.5          # Faktor für die Standardabweichung zur Outlier-Erkennung
min_distance_for_shift = 15       # Minimum DISTANCE, um als Shift klassifiziert zu werden

# Funktion zur verbesserten Klassifikation
def outlier_classification(group):
    group = group.sort_values('RECORDED_DATE')  # Daten sortieren
    distances = group['DISTANCE'].values

    # Initialisierung der Klassifikationen
    group['CLASSIFICATION'] = 'accurate'
    group['outlier'] = False

    # Schritt 1: Harte Grenze für Outliers anwenden
    for i in group.index:
        if group.loc[i, 'DISTANCE'] > hard_outlier_threshold:
            group.loc[i, 'CLASSIFICATION'] = 'outlier'
            group.loc[i, 'outlier'] = True

    # Schritt 2: Iterative Berechnung und Entfernung von Outliers
    remaining_points = group[group['CLASSIFICATION'] != 'outlier']
    while True:
        # Berechnung von rolling_mean und rolling_std basierend auf verbleibenden Punkten
        rolling_mean = pd.Series(remaining_points['DISTANCE'].values).rolling(window=4, center=True).mean()
        rolling_std = pd.Series(remaining_points['DISTANCE'].values).rolling(window=4, center=True).std()

        # Outliers anhand dynamischer Statistiken erkennen
        new_outliers = []
        for i in remaining_points.index:
            current_distance = group.loc[i, 'DISTANCE']
            mean_val = rolling_mean.iloc[remaining_points.index.get_loc(i)]
            std_val = rolling_std.iloc[remaining_points.index.get_loc(i)]
            diff = abs(current_distance - mean_val)

            if diff > outlier_absolute_threshold or (std_val > 0 and diff > outlier_multiplier * std_val):
                new_outliers.append(i)

        # Wenn keine neuen Outliers gefunden wurden, breche die Schleife ab
        if not new_outliers:
            break

        # Markiere neue Outliers und entferne sie aus den verbleibenden Punkten
        group.loc[new_outliers, 'CLASSIFICATION'] = 'outlier'
        group.loc[new_outliers, 'outlier'] = True
        remaining_points = group[group['CLASSIFICATION'] != 'outlier']

    # Schritt 3: Am Ende alle Punkte mit DISTANCE < 15m als accurate markieren
    for i in group.index:
        if group.loc[i, 'DISTANCE'] < min_distance_for_shift:
            group.loc[i, 'CLASSIFICATION'] = 'accurate'

    return group

# Gruppierte Anwendung pro Location
new_df_ContainerActivities = (
    new_df_ContainerActivities.groupby('LOCATION_ID', group_keys=False)
    .apply(outlier_classification)
)

new_df_ContainerActivities

/var/folders/hp/y7wgcxwd5s7ff9cjh270v5v40000gp/T/ipykernel_12607/3132579840.py:60: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(outlier_classification)


,TRANSACTION_ID,CONTAINER_ID,SLIDER_LEVEL,RECORDED_AT,COMMENT,IS_EMPTIED,LATITUDE,LONGITUDE,UPDATED_AT,LOCATION_ID,GEO_LAT,GEO_LON,LATITUDE_DIFF,LONGITUDE_DIFF,DISTANCE,RECORDED_DATE,CLASSIFICATION,outlier
135,217,1001101101100011111,50,2023-09-27 09:35:00,NaN,1,49.404942,7.726891,NaN,100110110110001,49.404778,7.726918,0.000164,-0.000027,18.344643,2023-09-27,accurate,False
615,1175,1001101101100011111,40,2023-10-05 09:24:27,NaN,1,49.404931,7.726842,NaN,100110110110001,49.404778,7.726918,0.000153,-0.000076,17.887946,2023-10-05,accurate,False
1147,2353,1001101101100011111,50,2023-10-11 10:04:28,NaN,1,49.404906,7.726879,NaN,100110110110001,49.404778,7.726918,0.000128,-0.000039,14.514515,2023-10-11,accurate,False
1729,3514,1001101101100011111,70,2023-10-17 10:16:43,NaN,1,49.404898,7.726916,NaN,100110110110001,49.404778,7.726918,0.000120,-0.000002,13.346898,2023-10-17,accurate,False
2248,4515,1001101101100011111,80,2023-10-24 13:03:25,NaN,1,49.404935,7.726907,NaN,100110110110001,49.404778,7.726918,0.000157,-0.000011,17.479400,2023-10-24,accurate,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28384,53127,1001101102101301311,10,2024-08-02 08:24:33,NaN,0,49.483645,7.624015,2024-09-30 15:32:27.0,100110110210130,49.437512,7.531113,0.046133,0.092902,8466.666234,2024-08-02,outlier,True
28807,53880,1001101102101301111,50,2024-08-09 09:39:55,NaN,1,49.437382,7.530471,2024-09-30 15:32:27.0,100110110210130,49.437512,7.531113,-0.000130,-0.000642,48.771136,2024-08-09,accurate,False
29173,54691,1001101102101301111,100,2024-08-16 09:00:52,NaN,1,49.437395,7.530600,2024-09-30 15:32:27.0,100110110210130,49.437512,7.531113,-0.000117,-0.000513,39.411288,2024-08-16,accurate,False
29671,55521,1001101102101301112,0,2024-08-22 10:53:19,NaN,0,49.441571,7.563419,2024-08-22 09:08:44.0,100110110210130,49.437512,7.531113,0.004059,0.032306,2386.088913,2024-08-22,outlier,True


In [6]:
# Übersicht der Klassifikationen
classification_summary = new_df_ContainerActivities['CLASSIFICATION'].value_counts()
print("Classification Summary:")
print(classification_summary)

Classification Summary:
CLASSIFICATION
accurate    7149
outlier     2072
Name: count, dtype: int64


In [7]:
''''
location_ids = new_df_ContainerActivities['LOCATION_ID'].unique()

# Interaktive Funktion erstellen
def plot_location_interactive(location_id):
    # Daten filtern für die gewählte Location
    location_data = new_df_ContainerActivities[new_df_ContainerActivities['LOCATION_ID'] == location_id]
    location_data = location_data.sort_values('RECORDED_DATE')  # Chronologisch sortieren

    # Plot initialisieren
    plt.figure(figsize=(12, 6))

    # Verbindungen zeichnen
    for i in range(len(location_data) - 1):
        current_row = location_data.iloc[i]
        next_row = location_data.iloc[i + 1]

        # Verbinde nur accurate und temporary_shift
        if current_row['CLASSIFICATION'] in ['accurate', 'temporary_shift'] and next_row['CLASSIFICATION'] in ['accurate', 'temporary_shift']:
            line_color = 'blue'
            
            # Zeichne die Linie
            plt.plot(
                [current_row['RECORDED_DATE'], next_row['RECORDED_DATE']],
                [current_row['DISTANCE'], next_row['DISTANCE']],
                color=line_color,
                linestyle='-', linewidth=1
            )

    # Punkte zeichnen
    for CLASSIFICATION, color, marker in [('accurate', 'blue', 'o'), 
                                           ('temporary_shift', 'blue', 'o'),
                                           ('outlier', 'red', 'x')]:
        filtered_data = location_data[location_data['CLASSIFICATION'] == CLASSIFICATION]
        plt.scatter(
            filtered_data['RECORDED_DATE'],
            filtered_data['DISTANCE'],
            color=color,
            label=CLASSIFICATION.capitalize(),
            marker=marker,
            s=100  # Punktgröße
        )

    # Titel und Achsenbeschriftungen
    plt.title(f"Distance Over Time for Location {location_id}")
    plt.xlabel("Date")
    plt.ylabel("Distance (meters)")
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


# Interaktives Dropdown-Menü
interactive_plot = interactive(plot_location_interactive, location_id=location_ids)
display(interactive_plot)'''

'\'\nlocation_ids = new_df_ContainerActivities[\'LOCATION_ID\'].unique()\n\n# Interaktive Funktion erstellen\ndef plot_location_interactive(location_id):\n    # Daten filtern für die gewählte Location\n    location_data = new_df_ContainerActivities[new_df_ContainerActivities[\'LOCATION_ID\'] == location_id]\n    location_data = location_data.sort_values(\'RECORDED_DATE\')  # Chronologisch sortieren\n\n    # Plot initialisieren\n    plt.figure(figsize=(12, 6))\n\n    # Verbindungen zeichnen\n    for i in range(len(location_data) - 1):\n        current_row = location_data.iloc[i]\n        next_row = location_data.iloc[i + 1]\n\n        # Verbinde nur accurate und temporary_shift\n        if current_row[\'CLASSIFICATION\'] in [\'accurate\', \'temporary_shift\'] and next_row[\'CLASSIFICATION\'] in [\'accurate\', \'temporary_shift\']:\n            line_color = \'blue\'\n            \n            # Zeichne die Linie\n            plt.plot(\n                [current_row[\'RECORDED_DATE\'], nex

In [8]:
scaling_factor_span = 0.3       # Faktor für shift_span (30% der Spannweite)
scaling_factor_min_distance = 0.3  # Faktor für min_distance_for_shift (30% des Mittelwerts)
min_shift_threshold = 35        # Mindestwert für min_distance_for_shift
min_points_for_shift = 3        # Mindestanzahl von Punkten für einen Shift
rolling_window_size = 3         # Fenstergröße für Rolling Mean/Median

def temporary_shift_detection(group):
    # Entferne Outliers temporär
    non_outliers = group[group['CLASSIFICATION'] != 'outlier'].copy()
    non_outliers['CLASSIFICATION'] = 'accurate'  # Standardmäßig accurate setzen

    # Dynamische Parameter berechnen
    location_range = non_outliers['DISTANCE'].max() - non_outliers['DISTANCE'].min()
    location_mean = non_outliers['DISTANCE'].mean()

    # Dynamischer shift_span
    shift_span = max(10, scaling_factor_span * location_range)  # Mindestwert 10
    # Dynamisches min_distance_for_shift
    min_distance_for_shift = max(min_shift_threshold, scaling_factor_min_distance * location_mean)

    # Schritt 1: Rolling Median berechnen
    non_outliers['rolling_median'] = non_outliers['DISTANCE'].rolling(window=rolling_window_size, center=True).median()

    # Schritt 2: Temporary Shifts erkennen
    potential_shift_group = []
    for i in non_outliers.index:
        distance = non_outliers.loc[i, 'DISTANCE']
        rolling_median = non_outliers.loc[i, 'rolling_median']

        # Bedingung: Punkt muss innerhalb des dynamischen Shift-Spans liegen
        if distance >= min_distance_for_shift and abs(distance - rolling_median) <= shift_span:
            potential_shift_group.append(i)
        else:
            # Prüfe die aktuelle Gruppe, wenn Bedingung nicht erfüllt
            if len(potential_shift_group) >= min_points_for_shift:
                # Markiere die Gruppe als Temporary Shift
                non_outliers.loc[potential_shift_group, 'CLASSIFICATION'] = 'temporary_shift'
            # Gruppe zurücksetzen
            potential_shift_group = []

    # Prüfe letzte Gruppe nach der Schleife
    if len(potential_shift_group) >= min_points_for_shift:
        non_outliers.loc[potential_shift_group, 'CLASSIFICATION'] = 'temporary_shift'

    # Schritt 3: Einzelne Punkte zwischen zwei Shift-Phasen prüfen
    indices = non_outliers.index
    for i in range(1, len(indices) - 1):
        prev_idx = indices[i - 1]
        curr_idx = indices[i]
        next_idx = indices[i + 1]

        # Prüfe, ob vorher und nachher Temporary Shifts sind
        if (
            non_outliers.loc[prev_idx, 'CLASSIFICATION'] == 'temporary_shift' and
            non_outliers.loc[next_idx, 'CLASSIFICATION'] == 'temporary_shift' and
            non_outliers.loc[curr_idx, 'CLASSIFICATION'] == 'accurate'
        ):
            prev_value = non_outliers.loc[prev_idx, 'DISTANCE']
            next_value = non_outliers.loc[next_idx, 'DISTANCE']

            # Prüfe, ob die beiden äußeren Punkte auf einem ähnlichen Niveau sind
            if abs(prev_value - next_value) <= shift_span:
                # Markiere den mittleren Punkt als temporary_shift
                non_outliers.loc[curr_idx, 'CLASSIFICATION'] = 'temporary_shift'

    # Schritt 4: Aktualisiere die ursprüngliche Gruppe
    group.update(non_outliers)
    return group

# Anwendung auf die Daten
new_df_ContainerActivities = (
    new_df_ContainerActivities.groupby('LOCATION_ID', group_keys=False)
    .apply(temporary_shift_detection)
)

# Entferne Hilfsspalten, falls sie nicht mehr benötigt werden
new_df_ContainerActivities.drop(columns=['rolling_median'], inplace=True, errors='ignore')

new_df_ContainerActivities


/var/folders/hp/y7wgcxwd5s7ff9cjh270v5v40000gp/T/ipykernel_12607/723555026.py:73: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(temporary_shift_detection)


,TRANSACTION_ID,CONTAINER_ID,SLIDER_LEVEL,RECORDED_AT,COMMENT,IS_EMPTIED,LATITUDE,LONGITUDE,UPDATED_AT,LOCATION_ID,GEO_LAT,GEO_LON,LATITUDE_DIFF,LONGITUDE_DIFF,DISTANCE,RECORDED_DATE,CLASSIFICATION,outlier
135,217,1001101101100011111,50,2023-09-27 09:35:00,NaN,1,49.404942,7.726891,NaN,100110110110001,49.404778,7.726918,0.000164,-0.000027,18.344643,2023-09-27,accurate,False
615,1175,1001101101100011111,40,2023-10-05 09:24:27,NaN,1,49.404931,7.726842,NaN,100110110110001,49.404778,7.726918,0.000153,-0.000076,17.887946,2023-10-05,accurate,False
1147,2353,1001101101100011111,50,2023-10-11 10:04:28,NaN,1,49.404906,7.726879,NaN,100110110110001,49.404778,7.726918,0.000128,-0.000039,14.514515,2023-10-11,accurate,False
1729,3514,1001101101100011111,70,2023-10-17 10:16:43,NaN,1,49.404898,7.726916,NaN,100110110110001,49.404778,7.726918,0.000120,-0.000002,13.346898,2023-10-17,accurate,False
2248,4515,1001101101100011111,80,2023-10-24 13:03:25,NaN,1,49.404935,7.726907,NaN,100110110110001,49.404778,7.726918,0.000157,-0.000011,17.479400,2023-10-24,accurate,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28384,53127,1001101102101301248,10,2024-08-02 08:24:33,NaN,0,49.483645,7.624015,2024-09-30 15:32:27.0,100110110210130,49.437512,7.531113,0.046133,0.092902,8466.666234,2024-08-02,outlier,True
28807,53880,1001101102101301120,50,2024-08-09 09:39:55,NaN,1,49.437382,7.530471,2024-09-30 15:32:27.0,100110110210130,49.437512,7.531113,-0.000130,-0.000642,48.771136,2024-08-09,temporary_shift,False
29173,54691,1001101102101301120,100,2024-08-16 09:00:52,NaN,1,49.437395,7.530600,2024-09-30 15:32:27.0,100110110210130,49.437512,7.531113,-0.000117,-0.000513,39.411288,2024-08-16,accurate,False
29671,55521,1001101102101301120,0,2024-08-22 10:53:19,NaN,0,49.441571,7.563419,2024-08-22 09:08:44.0,100110110210130,49.437512,7.531113,0.004059,0.032306,2386.088913,2024-08-22,outlier,True


In [9]:
# Übersicht der Klassifikationen
classification_summary = new_df_ContainerActivities['CLASSIFICATION'].value_counts()
print("Classification Summary:")
print(classification_summary)

Classification Summary:
CLASSIFICATION
accurate           5741
outlier            2072
temporary_shift    1408
Name: count, dtype: int64


In [10]:
''''
location_ids = new_df_ContainerActivities['LOCATION_ID'].unique()

# Funktion, um die Grafik für eine Location ohne Ausreißer zu erstellen
def plot_location_no_outliers(location_id):
    # Filtere Daten für die ausgewählte Location
    location_data = new_df_ContainerActivities[new_df_ContainerActivities['LOCATION_ID'] == location_id]
    location_data = location_data.sort_values('RECORDED_DATE')
    
    # Plot initialisieren
    plt.figure(figsize=(12, 6))
    
    # Entferne Ausreißer
    location_data = location_data[location_data['CLASSIFICATION'] != 'outlier']
    location_data = location_data[location_data['outlier'] == False]

    # Verbindungen zeichnen
    for i in range(len(location_data) - 1):
        current_row = location_data.iloc[i]
        next_row = location_data.iloc[i + 1]

        # Verbinde nur accurate und temporary_shift
        if current_row['CLASSIFICATION'] in ['accurate', 'temporary_shift'] and next_row['CLASSIFICATION'] in ['accurate', 'temporary_shift']:
            # Setze die Linienfarbe
            if current_row['CLASSIFICATION'] == 'accurate' or next_row['CLASSIFICATION'] == 'accurate':
                line_color = 'green'  # Eine Seite accurate
            else:
                line_color = 'orange'   # Beide Seiten temporary_shift

    # Zeichne die Linie
            plt.plot(
                [current_row['RECORDED_DATE'], next_row['RECORDED_DATE']],
                [current_row['DISTANCE'], next_row['DISTANCE']],
                color=line_color,
                linestyle='-', linewidth=1
            )

    # Punkte zeichnen
    for CLASSIFICATION, color, marker in [('accurate', 'green', 'o'), 
                                           ('temporary_shift', 'orange', 'o')]:
        filtered_data = location_data[location_data['CLASSIFICATION'] == CLASSIFICATION]
        plt.scatter(
            filtered_data['RECORDED_DATE'],
            filtered_data['DISTANCE'],
            color=color,
            label=CLASSIFICATION.capitalize(),
            marker=marker,
            s=100  # Punktgröße
        )

    # Titel und Achsenbeschriftungen
    plt.title(f"Distance Over Time for Location {location_id}")
    plt.xlabel("Date")
    plt.ylabel("Distance (meters)")
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Interaktives Dropdown-Menü
interactive_plot_no_outliers = interactive(plot_location_no_outliers, location_id=location_ids)
display(interactive_plot_no_outliers)
'''

'\'\nlocation_ids = new_df_ContainerActivities[\'LOCATION_ID\'].unique()\n\n# Funktion, um die Grafik für eine Location ohne Ausreißer zu erstellen\ndef plot_location_no_outliers(location_id):\n    # Filtere Daten für die ausgewählte Location\n    location_data = new_df_ContainerActivities[new_df_ContainerActivities[\'LOCATION_ID\'] == location_id]\n    location_data = location_data.sort_values(\'RECORDED_DATE\')\n    \n    # Plot initialisieren\n    plt.figure(figsize=(12, 6))\n    \n    # Entferne Ausreißer\n    location_data = location_data[location_data[\'CLASSIFICATION\'] != \'outlier\']\n    location_data = location_data[location_data[\'outlier\'] == False]\n\n    # Verbindungen zeichnen\n    for i in range(len(location_data) - 1):\n        current_row = location_data.iloc[i]\n        next_row = location_data.iloc[i + 1]\n\n        # Verbinde nur accurate und temporary_shift\n        if current_row[\'CLASSIFICATION\'] in [\'accurate\', \'temporary_shift\'] and next_row[\'CLASSIFI

# Cleaning coordinates

In [11]:
def calculate_shifted_coords_with_outliers(location_group):
    """
    Gruppiert `temporary_shift`-Punkte basierend auf aufeinanderfolgender Reihenfolge nach Sortierung
    und berücksichtigt Outlier zwischen zwei Phasen.
    """
    # Nur temporär verschobene Punkte und Outlier betrachten
    temp_shifted = location_group[location_group['CLASSIFICATION'] == 'temporary_shift']
    outliers = location_group[location_group['CLASSIFICATION'] == 'outlier']

    # Prüfen, ob temporär verschobene Punkte existieren
    if temp_shifted.empty:
        return location_group

    # Originalindex speichern und Daten nach Datum sortieren
    temp_shifted = temp_shifted.sort_values('RECORDED_DATE')
    temp_shifted['original_index'] = temp_shifted.index  # Speichere Originalindex
    temp_shifted = temp_shifted.reset_index(drop=True)  # Index zurücksetzen

    # Identifiziere neue Phasen basierend auf Lücken im Index
    temp_shifted['shift_phase'] = (temp_shifted.index.to_series().diff() > 1).cumsum()

    # Durchschnittswerte für jede Phase berechnen
    phase_means = temp_shifted.groupby('shift_phase')[['LATITUDE', 'LONGITUDE']].mean()

    # Durchschnittswerte auf die jeweilige Phase anwenden
    for phase_id, phase_coords in phase_means.iterrows():
        phase_indices = temp_shifted[temp_shifted['shift_phase'] == phase_id]['original_index']
        location_group.loc[phase_indices, 'NEW_LAT'] = phase_coords['LATITUDE']
        location_group.loc[phase_indices, 'NEW_LON'] = phase_coords['LONGITUDE']

    # Outlier zwischen Phasen identifizieren
    for index, outlier in outliers.iterrows():
        outlier_date = outlier['RECORDED_DATE']
        
        # Benachbarte Phasen basierend auf Datum finden
        before_phase = temp_shifted[temp_shifted['RECORDED_DATE'] < outlier_date].tail(1)
        after_phase = temp_shifted[temp_shifted['RECORDED_DATE'] > outlier_date].head(1)

        if not before_phase.empty and not after_phase.empty:
            # Phasen-IDs und Mittelwerte der Nachbarn ermitteln
            before_phase_id = before_phase['shift_phase'].iloc[0]
            after_phase_id = after_phase['shift_phase'].iloc[0]
            before_coords = phase_means.loc[before_phase_id]
            after_coords = phase_means.loc[after_phase_id]

            # Mittelwert der benachbarten Phasen berechnen
            mean_lat = (before_coords['LATITUDE'] + after_coords['LATITUDE']) / 2
            mean_lon = (before_coords['LONGITUDE'] + after_coords['LONGITUDE']) / 2

            # Outlier anpassen
            location_group.loc[index, 'NEW_LAT'] = mean_lat
            location_group.loc[index, 'NEW_LON'] = mean_lon
            location_group.loc[index, 'CLASSIFICATION'] = 'temporary_shift'

    return location_group


# Anwendung auf alle LOCATION_IDs
def update_coordinates_with_outliers(df):
    # Spalte für neue Koordinaten initialisieren
    df['NEW_LAT'] = np.nan
    df['NEW_LON'] = np.nan

    # Koordinaten für accurate setzen
    df.loc[df['CLASSIFICATION'] == 'accurate', 'NEW_LAT'] = df['GEO_LAT']
    df.loc[df['CLASSIFICATION'] == 'accurate', 'NEW_LON'] = df['GEO_LON']

    # Berechnung für jede Location-ID
    df = df.groupby('LOCATION_ID', group_keys=False).apply(calculate_shifted_coords_with_outliers)
    
    remaining_outliers = (df['CLASSIFICATION'] == 'outlier') & (df['NEW_LAT'].isna()) & (df['NEW_LON'].isna())
    df.loc[remaining_outliers, 'NEW_LAT'] = df.loc[remaining_outliers, 'GEO_LAT']
    df.loc[remaining_outliers, 'NEW_LON'] = df.loc[remaining_outliers, 'GEO_LON']

    # Spalte shift_phase entfernen, falls sie nicht mehr benötigt wird
    df.drop(columns=['shift_phase', 'original_index'], inplace=True, errors='ignore')

    return df


# Update Coordinates
new_df_ContainerActivities = update_coordinates_with_outliers(new_df_ContainerActivities)

# Resultat anzeigen
new_df_ContainerActivities[['LOCATION_ID', 'CLASSIFICATION', 'LATITUDE', 'LONGITUDE', 'GEO_LAT', 'GEO_LON', 'DISTANCE', 'NEW_LAT', 'NEW_LON']]


/var/folders/hp/y7wgcxwd5s7ff9cjh270v5v40000gp/T/ipykernel_12607/2781496985.py:69: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('LOCATION_ID', group_keys=False).apply(calculate_shifted_coords_with_outliers)


,LOCATION_ID,CLASSIFICATION,LATITUDE,LONGITUDE,GEO_LAT,GEO_LON,DISTANCE,NEW_LAT,NEW_LON
135,100110110110001,accurate,49.404942,7.726891,49.404778,7.726918,18.344643,49.404778,7.726918
615,100110110110001,accurate,49.404931,7.726842,49.404778,7.726918,17.887946,49.404778,7.726918
1147,100110110110001,accurate,49.404906,7.726879,49.404778,7.726918,14.514515,49.404778,7.726918
1729,100110110110001,accurate,49.404898,7.726916,49.404778,7.726918,13.346898,49.404778,7.726918
2248,100110110110001,accurate,49.404935,7.726907,49.404778,7.726918,17.479400,49.404778,7.726918
...,...,...,...,...,...,...,...,...,...
28384,100110110210130,temporary_shift,49.483645,7.624015,49.437512,7.531113,8466.666234,49.437560,7.528855
28807,100110110210130,temporary_shift,49.437382,7.530471,49.437512,7.531113,48.771136,49.437560,7.528855
29173,100110110210130,accurate,49.437395,7.530600,49.437512,7.531113,39.411288,49.437512,7.531113
29671,100110110210130,outlier,49.441571,7.563419,49.437512,7.531113,2386.088913,49.437512,7.531113


In [12]:
new_df_ContainerActivities = new_df_ContainerActivities.dropna(subset=['NEW_LAT', 'NEW_LON', 'GEO_LAT', 'GEO_LON'])
new_df_ContainerActivities['NEW_DISTANCE'] = new_df_ContainerActivities.apply(lambda x: geodesic((x['NEW_LAT'], x['NEW_LON']), (x['GEO_LAT'], x['GEO_LON'])).m, axis=1)
new_df_ContainerActivities[['LOCATION_ID', 'CLASSIFICATION', 'LATITUDE', 'LONGITUDE', 'GEO_LAT', 'GEO_LON','DISTANCE' ,'NEW_LAT', 'NEW_LON', 'NEW_DISTANCE']]

,LOCATION_ID,CLASSIFICATION,LATITUDE,LONGITUDE,GEO_LAT,GEO_LON,DISTANCE,NEW_LAT,NEW_LON,NEW_DISTANCE
135,100110110110001,accurate,49.404942,7.726891,49.404778,7.726918,18.344643,49.404778,7.726918,0.000000
615,100110110110001,accurate,49.404931,7.726842,49.404778,7.726918,17.887946,49.404778,7.726918,0.000000
1147,100110110110001,accurate,49.404906,7.726879,49.404778,7.726918,14.514515,49.404778,7.726918,0.000000
1729,100110110110001,accurate,49.404898,7.726916,49.404778,7.726918,13.346898,49.404778,7.726918,0.000000
2248,100110110110001,accurate,49.404935,7.726907,49.404778,7.726918,17.479400,49.404778,7.726918,0.000000
...,...,...,...,...,...,...,...,...,...,...
28384,100110110210130,temporary_shift,49.483645,7.624015,49.437512,7.531113,8466.666234,49.437560,7.528855,163.867643
28807,100110110210130,temporary_shift,49.437382,7.530471,49.437512,7.531113,48.771136,49.437560,7.528855,163.867643
29173,100110110210130,accurate,49.437395,7.530600,49.437512,7.531113,39.411288,49.437512,7.531113,0.000000
29671,100110110210130,outlier,49.441571,7.563419,49.437512,7.531113,2386.088913,49.437512,7.531113,0.000000


In [13]:
''''
location_ids = new_df_ContainerActivities['LOCATION_ID'].unique()

# Funktion, um die Grafik für eine Location zu erstellen
def plot_location_no_outliers(location_id):
    # Filtere Daten für die ausgewählte Location
    location_data = new_df_ContainerActivities[new_df_ContainerActivities['LOCATION_ID'] == location_id]
    location_data = location_data.sort_values('RECORDED_DATE')

    # Plot initialisieren
    plt.figure(figsize=(12, 6))

    # Einmalige Labels verfolgen
    plotted_labels = set()

    # Verbindungen zeichnen
    for i in range(len(location_data) - 1):
        current_row = location_data.iloc[i]
        next_row = location_data.iloc[i + 1]

        # Verbinde accurate, outlier und temporary_shift
        if current_row['CLASSIFICATION'] in ['accurate', 'outlier', 'temporary_shift'] and next_row['CLASSIFICATION'] in ['accurate', 'outlier', 'temporary_shift']:
            # Setze die Linienfarbe
            if current_row['CLASSIFICATION'] in ['accurate', 'outlier'] or next_row['CLASSIFICATION'] in ['accurate', 'outlier']:
                line_color = 'green'  # Eine Seite accurate oder outlier
                label = "Accurate"
            else:
                line_color = 'orange'  # Beide Seiten temporary_shift
                label = "Temporary Shift"

            # Zeichne die Linie und füge das Label nur einmal hinzu
            plt.plot(
                [current_row['RECORDED_DATE'], next_row['RECORDED_DATE']],
                [current_row['NEW_DISTANCE'], next_row['NEW_DISTANCE']],
                color=line_color,
                linestyle='-',
                linewidth=2,
                label=label if label not in plotted_labels else ""
            )
            plotted_labels.add(label)

    if "Original Distance" not in plotted_labels:
        # Linie für alle Punkte (inklusive Outlier) plotten
        plt.plot(
            location_data['RECORDED_DATE'],
            location_data['DISTANCE'],
            linestyle='--',
            color='purple',
            linewidth=2,
            label="Original Distance"
        )
    
        # Outlier als rotes x plotten
        outliers = location_data[location_data['outlier'] == True]
        plt.scatter(
            outliers['RECORDED_DATE'],
            outliers['DISTANCE'],
            color='red',
            marker='x',
            label="Outlier"
        )
    
        # Hinzufügen der Legende für die Original-Distanz
        plotted_labels.add("Original Distance")
        plotted_labels.add("Outlier")

    # Titel und Achsenbeschriftungen
    plt.title(f"Distance Over Time for Location {location_id}")
    plt.xlabel("Date")
    plt.ylabel("Distance (meters)")
    plt.xticks(rotation=45)
    plt.grid(True)
    
    # Legende hinzufügen
    plt.legend()
    plt.tight_layout()
    plt.show()

# Interaktives Dropdown-Menü
interactive_plot_no_outliers = interactive(plot_location_no_outliers, location_id=location_ids)
display(interactive_plot_no_outliers)
'''

'\'\nlocation_ids = new_df_ContainerActivities[\'LOCATION_ID\'].unique()\n\n# Funktion, um die Grafik für eine Location zu erstellen\ndef plot_location_no_outliers(location_id):\n    # Filtere Daten für die ausgewählte Location\n    location_data = new_df_ContainerActivities[new_df_ContainerActivities[\'LOCATION_ID\'] == location_id]\n    location_data = location_data.sort_values(\'RECORDED_DATE\')\n\n    # Plot initialisieren\n    plt.figure(figsize=(12, 6))\n\n    # Einmalige Labels verfolgen\n    plotted_labels = set()\n\n    # Verbindungen zeichnen\n    for i in range(len(location_data) - 1):\n        current_row = location_data.iloc[i]\n        next_row = location_data.iloc[i + 1]\n\n        # Verbinde accurate, outlier und temporary_shift\n        if current_row[\'CLASSIFICATION\'] in [\'accurate\', \'outlier\', \'temporary_shift\'] and next_row[\'CLASSIFICATION\'] in [\'accurate\', \'outlier\', \'temporary_shift\']:\n            # Setze die Linienfarbe\n            if current_ro

# Save the cleaned data

In [14]:
#new_df_ContainerActivities = new_df_ContainerActivities[['LOCATION_ID', 'RECORDED_AT', 'NEW_LAT', 'NEW_LON', 'NEW_DISTANCE']]

In [15]:
folder_path = '/Users/annele/PycharmProjects/Glas-O-Mat/data'
df_ContainerActivities = pd.read_csv(os.path.join(folder_path, 'modified_data/ContainerActivities_filtered.csv')) 
df_Locations = pd.read_csv(os.path.join(folder_path, 'Locations.csv'))

In [16]:
# add latitude and longitude to ContainerActivities by merging with Locations
df_Locations['LOCATION_ID'] = df_Locations['LOCATION_ID'].astype(str)
df_ContainerActivities['LOCATION_ID'] = df_ContainerActivities['LOCATION_ID'].astype(str)
df_ContainerActivities = pd.merge(df_ContainerActivities, df_Locations[['LOCATION_ID', 'GEO_LAT', 'GEO_LON']], on='LOCATION_ID', how='left')
df_ContainerActivities

,TRANSACTION_ID,CONTAINER_ID,SLIDER_LEVEL,CV_LEVEL,SENSOR_LEVEL,IMAGE_ID,PHONE_ID,RECORDED_AT,COMMENT,IS_EMPTIED,LATITUDE,LONGITUDE,CREATED_AT,UPDATED_AT,ROUTE_ID,LOCATION_ID,GEO_LAT,GEO_LON
0,1,1001101101100271111,30,NaN,NaN,1001101101100271111/2023-09-28T06:36:48.022095...,TP1A.220624.014_2,2023-09-26 06:36:48.0,NaN,1,49.455948,7.791700,2023-10-13 11:18:58.0,NaN,NaN,100110110110027,49.456130,7.791667
1,2,1001101101100271211,20,NaN,NaN,1001101101100271211/2023-09-28T06:36:05.355751...,TP1A.220624.014_2,2023-09-26 06:36:05.0,NaN,1,49.455948,7.791700,2023-10-13 11:18:58.0,NaN,NaN,100110110110027,49.456130,7.791667
2,3,1001101101100271311,10,NaN,NaN,1001101101100271311/2023-09-28T06:37:16.029149...,TP1A.220624.014_2,2023-09-26 06:37:16.0,NaN,0,49.455948,7.791700,2023-10-13 11:18:58.0,NaN,NaN,100110110110027,49.456130,7.791667
3,4,1001101101100191111,0,NaN,NaN,1001101101100191111/2023-09-28T06:51:15.117682...,TP1A.220624.014_2,2023-09-26 06:51:15.0,NaN,0,49.448544,7.769812,2023-10-13 11:18:58.0,NaN,NaN,100110110110019,49.448608,7.769743
4,5,1001101101100191211,10,NaN,NaN,1001101101100191211/2023-09-28T06:51:43.824174...,TP1A.220624.014_2,2023-09-26 06:51:44.0,NaN,0,49.448544,7.769812,2023-10-13 11:18:58.0,NaN,NaN,100110110110019,49.448608,7.769743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35916,61150,1001101101100531112,90,NaN,NaN,1001101101100531112/2024-10-17T14:47:10.767846...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:47:11.0,NaN,1,49.494200,49.494200,2024-10-17 12:47:32.0,2024-10-17 12:47:32.0,2024-10-17-1001101101-0,100110110110053,49.440841,7.708089
35917,61151,1001101101100531211,20,NaN,NaN,1001101101100531211/2024-10-17T14:46:43.691739...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:46:44.0,NaN,1,49.494200,49.494200,2024-10-17 12:47:33.0,2024-10-17 12:47:33.0,2024-10-17-1001101101-0,100110110110053,49.440841,7.708089
35918,61152,1001101101100531212,90,NaN,NaN,1001101101100531212/2024-10-17T14:46:54.523517...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:46:55.0,NaN,1,49.494200,49.494200,2024-10-17 12:47:33.0,2024-10-17 12:47:33.0,2024-10-17-1001101101-0,100110110110053,49.440841,7.708089
35919,61153,1001101101100531311,20,NaN,NaN,1001101101100531311/2024-10-17T14:47:19.611355...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:47:20.0,NaN,0,49.494200,49.494200,2024-10-17 12:47:34.0,2024-10-17 12:47:34.0,2024-10-17-1001101101-0,100110110110053,49.440841,7.708089


In [17]:
new_df_ContainerActivities['LOCATION_ID'] = new_df_ContainerActivities['LOCATION_ID'].astype(str)
new_df_ContainerActivities['DATE'] = pd.to_datetime(new_df_ContainerActivities['RECORDED_AT']).dt.date
df_ContainerActivities['DATE'] = pd.to_datetime(df_ContainerActivities['RECORDED_AT']).dt.date

# Merge basierend auf LOCATION_ID und DATE
df_ContainerActivities = pd.merge(
    df_ContainerActivities,
    new_df_ContainerActivities[['LOCATION_ID', 'CLASSIFICATION', 'outlier', 'DATE', 'DISTANCE', 'NEW_LAT', 'NEW_LON', 'NEW_DISTANCE']],
    on=['LOCATION_ID', 'DATE'],
    how='left'  # Nutze 'left', um alle Zeilen aus dem alten DataFrame beizubehalten
)
df_ContainerActivities

,TRANSACTION_ID,CONTAINER_ID,SLIDER_LEVEL,CV_LEVEL,SENSOR_LEVEL,IMAGE_ID,PHONE_ID,RECORDED_AT,COMMENT,IS_EMPTIED,...,LOCATION_ID,GEO_LAT,GEO_LON,DATE,CLASSIFICATION,outlier,DISTANCE,NEW_LAT,NEW_LON,NEW_DISTANCE
0,1,1001101101100271111,30,NaN,NaN,1001101101100271111/2023-09-28T06:36:48.022095...,TP1A.220624.014_2,2023-09-26 06:36:48.0,NaN,1,...,100110110110027,49.456130,7.791667,2023-09-26,accurate,False,20.382686,49.456130,7.791667,0.0
1,2,1001101101100271211,20,NaN,NaN,1001101101100271211/2023-09-28T06:36:05.355751...,TP1A.220624.014_2,2023-09-26 06:36:05.0,NaN,1,...,100110110110027,49.456130,7.791667,2023-09-26,accurate,False,20.382686,49.456130,7.791667,0.0
2,3,1001101101100271311,10,NaN,NaN,1001101101100271311/2023-09-28T06:37:16.029149...,TP1A.220624.014_2,2023-09-26 06:37:16.0,NaN,0,...,100110110110027,49.456130,7.791667,2023-09-26,accurate,False,20.382686,49.456130,7.791667,0.0
3,4,1001101101100191111,0,NaN,NaN,1001101101100191111/2023-09-28T06:51:15.117682...,TP1A.220624.014_2,2023-09-26 06:51:15.0,NaN,0,...,100110110110019,49.448608,7.769743,2023-09-26,accurate,False,8.700529,49.448608,7.769743,0.0
4,5,1001101101100191211,10,NaN,NaN,1001101101100191211/2023-09-28T06:51:43.824174...,TP1A.220624.014_2,2023-09-26 06:51:44.0,NaN,0,...,100110110110019,49.448608,7.769743,2023-09-26,accurate,False,8.700529,49.448608,7.769743,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35916,61150,1001101101100531112,90,NaN,NaN,1001101101100531112/2024-10-17T14:47:10.767846...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:47:11.0,NaN,1,...,100110110110053,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,NaN,NaN,NaN
35917,61151,1001101101100531211,20,NaN,NaN,1001101101100531211/2024-10-17T14:46:43.691739...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:46:44.0,NaN,1,...,100110110110053,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,NaN,NaN,NaN
35918,61152,1001101101100531212,90,NaN,NaN,1001101101100531212/2024-10-17T14:46:54.523517...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:46:55.0,NaN,1,...,100110110110053,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,NaN,NaN,NaN
35919,61153,1001101101100531311,20,NaN,NaN,1001101101100531311/2024-10-17T14:47:19.611355...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:47:20.0,NaN,0,...,100110110110053,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_ContainerActivities[df_ContainerActivities[['NEW_LAT', 'NEW_LON', 'NEW_DISTANCE']].isna().any(axis=1)]

,TRANSACTION_ID,CONTAINER_ID,SLIDER_LEVEL,CV_LEVEL,SENSOR_LEVEL,IMAGE_ID,PHONE_ID,RECORDED_AT,COMMENT,IS_EMPTIED,...,LOCATION_ID,GEO_LAT,GEO_LON,DATE,CLASSIFICATION,outlier,DISTANCE,NEW_LAT,NEW_LON,NEW_DISTANCE
2907,5501,1001101101999991111,70,NaN,NaN,1001101101999991111/2023-11-05T09:18:28.218510...,TP1A.220624.014_6695359986a10fa0_2,2023-11-03 09:18:28.0,NaN,0,...,100110110199999,-1.000000,-1.000000,2023-11-03,NaN,NaN,NaN,NaN,NaN,NaN
2908,5502,1001101101999991211,50,NaN,NaN,1001101101999991211/2023-11-05T09:18:38.283725...,TP1A.220624.014_6695359986a10fa0_2,2023-11-03 09:18:38.0,NaN,0,...,100110110199999,-1.000000,-1.000000,2023-11-03,NaN,NaN,NaN,NaN,NaN,NaN
2909,5503,1001101101999991311,60,NaN,NaN,1001101101999991311/2023-11-05T09:18:14.720172...,TP1A.220624.014_6695359986a10fa0_2,2023-11-03 09:18:15.0,NaN,1,...,100110110199999,-1.000000,-1.000000,2023-11-03,NaN,NaN,NaN,NaN,NaN,NaN
3085,5695,1001101101999991111,110,NaN,NaN,1001101101999991111/2023-11-08T08:35:13.399861...,TP1A.220624.014_6695359986a10fa0_2,2023-11-06 08:35:13.0,NaN,1,...,100110110199999,-1.000000,-1.000000,2023-11-06,NaN,NaN,NaN,NaN,NaN,NaN
3086,5696,1001101101999991211,100,NaN,NaN,1001101101999991211/2023-11-08T08:35:20.515285...,TP1A.220624.014_6695359986a10fa0_2,2023-11-06 08:35:21.0,NaN,1,...,100110110199999,-1.000000,-1.000000,2023-11-06,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35916,61150,1001101101100531112,90,NaN,NaN,1001101101100531112/2024-10-17T14:47:10.767846...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:47:11.0,NaN,1,...,100110110110053,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,NaN,NaN,NaN
35917,61151,1001101101100531211,20,NaN,NaN,1001101101100531211/2024-10-17T14:46:43.691739...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:46:44.0,NaN,1,...,100110110110053,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,NaN,NaN,NaN
35918,61152,1001101101100531212,90,NaN,NaN,1001101101100531212/2024-10-17T14:46:54.523517...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:46:55.0,NaN,1,...,100110110110053,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,NaN,NaN,NaN
35919,61153,1001101101100531311,20,NaN,NaN,1001101101100531311/2024-10-17T14:47:19.611355...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:47:20.0,NaN,0,...,100110110110053,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
''''
these values are:
- placeholders
- locations that are not in the Location.csv
- locations with inaccurate coordinates
'''

"'\nthese values are:\n- placeholders\n- locations that are not in the Location.csv\n- locations with inaccurate coordinates\n"

# Clean innacurate coordinates

In [20]:
df_ContainerActivities['INACCURATE_COORDINATES'] = False

# Bedingung für Werte außerhalb des Koordinatenbereichs
invalid_coords = (
    (df_ContainerActivities['LATITUDE'] < 47) | 
    (df_ContainerActivities['LATITUDE'] > 55) | 
    (df_ContainerActivities['LONGITUDE'] < 6) | 
    (df_ContainerActivities['LONGITUDE'] > 15)
)

# Betroffene Werte mit GEO_LAT und GEO_LON überschreiben
df_ContainerActivities.loc[invalid_coords, 'NEW_LAT'] = df_ContainerActivities.loc[invalid_coords, 'GEO_LAT']
df_ContainerActivities.loc[invalid_coords, 'NEW_LON'] = df_ContainerActivities.loc[invalid_coords, 'GEO_LON']
df_ContainerActivities.loc[invalid_coords, 'NEW_DISTANCE'] = 0
df_ContainerActivities.loc[invalid_coords, 'INACCURATE_COORDINATES'] = True

df_ContainerActivities[invalid_coords]


,TRANSACTION_ID,CONTAINER_ID,SLIDER_LEVEL,CV_LEVEL,SENSOR_LEVEL,IMAGE_ID,PHONE_ID,RECORDED_AT,COMMENT,IS_EMPTIED,...,GEO_LAT,GEO_LON,DATE,CLASSIFICATION,outlier,DISTANCE,NEW_LAT,NEW_LON,NEW_DISTANCE,INACCURATE_COORDINATES
27038,50623,1001101102100771111,70,NaN,NaN,1001101102100771111/2024-07-14T18:40:51.822636...,UPB3.230519.006_5c4ed86666236a3d,2024-07-14 18:40:52.0,NaN,1,...,49.578750,7.663861,2024-07-14,NaN,NaN,NaN,49.578750,7.663861,0.0,True
27783,51380,1001101101100691111,10,NaN,NaN,1001101101100691111/2024-07-22T06:57:51.857140...,UP1A.231005.007_feb67a335a9479a4,2024-07-22 06:57:52.0,NaN,1,...,49.447244,7.772618,2024-07-22,NaN,NaN,NaN,49.447244,7.772618,0.0,True
27784,51381,1001101101100691112,30,NaN,NaN,1001101101100691112/2024-07-22T06:58:01.494934...,UP1A.231005.007_feb67a335a9479a4,2024-07-22 06:58:01.0,NaN,1,...,49.447244,7.772618,2024-07-22,NaN,NaN,NaN,49.447244,7.772618,0.0,True
27785,51382,1001101101100691211,20,NaN,NaN,1001101101100691211/2024-07-22T06:58:09.333789...,UP1A.231005.007_feb67a335a9479a4,2024-07-22 06:58:09.0,NaN,1,...,49.447244,7.772618,2024-07-22,NaN,NaN,NaN,49.447244,7.772618,0.0,True
27786,51383,1001101101100691212,20,NaN,NaN,1001101101100691212/2024-07-22T06:58:17.138790...,UP1A.231005.007_feb67a335a9479a4,2024-07-22 06:58:17.0,NaN,1,...,49.447244,7.772618,2024-07-22,NaN,NaN,NaN,49.447244,7.772618,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35916,61150,1001101101100531112,90,NaN,NaN,1001101101100531112/2024-10-17T14:47:10.767846...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:47:11.0,NaN,1,...,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True
35917,61151,1001101101100531211,20,NaN,NaN,1001101101100531211/2024-10-17T14:46:43.691739...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:46:44.0,NaN,1,...,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True
35918,61152,1001101101100531212,90,NaN,NaN,1001101101100531212/2024-10-17T14:46:54.523517...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:46:55.0,NaN,1,...,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True
35919,61153,1001101101100531311,20,NaN,NaN,1001101101100531311/2024-10-17T14:47:19.611355...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:47:20.0,NaN,0,...,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True


In [21]:
df_ContainerActivities

,TRANSACTION_ID,CONTAINER_ID,SLIDER_LEVEL,CV_LEVEL,SENSOR_LEVEL,IMAGE_ID,PHONE_ID,RECORDED_AT,COMMENT,IS_EMPTIED,...,GEO_LAT,GEO_LON,DATE,CLASSIFICATION,outlier,DISTANCE,NEW_LAT,NEW_LON,NEW_DISTANCE,INACCURATE_COORDINATES
0,1,1001101101100271111,30,NaN,NaN,1001101101100271111/2023-09-28T06:36:48.022095...,TP1A.220624.014_2,2023-09-26 06:36:48.0,NaN,1,...,49.456130,7.791667,2023-09-26,accurate,False,20.382686,49.456130,7.791667,0.0,False
1,2,1001101101100271211,20,NaN,NaN,1001101101100271211/2023-09-28T06:36:05.355751...,TP1A.220624.014_2,2023-09-26 06:36:05.0,NaN,1,...,49.456130,7.791667,2023-09-26,accurate,False,20.382686,49.456130,7.791667,0.0,False
2,3,1001101101100271311,10,NaN,NaN,1001101101100271311/2023-09-28T06:37:16.029149...,TP1A.220624.014_2,2023-09-26 06:37:16.0,NaN,0,...,49.456130,7.791667,2023-09-26,accurate,False,20.382686,49.456130,7.791667,0.0,False
3,4,1001101101100191111,0,NaN,NaN,1001101101100191111/2023-09-28T06:51:15.117682...,TP1A.220624.014_2,2023-09-26 06:51:15.0,NaN,0,...,49.448608,7.769743,2023-09-26,accurate,False,8.700529,49.448608,7.769743,0.0,False
4,5,1001101101100191211,10,NaN,NaN,1001101101100191211/2023-09-28T06:51:43.824174...,TP1A.220624.014_2,2023-09-26 06:51:44.0,NaN,0,...,49.448608,7.769743,2023-09-26,accurate,False,8.700529,49.448608,7.769743,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35916,61150,1001101101100531112,90,NaN,NaN,1001101101100531112/2024-10-17T14:47:10.767846...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:47:11.0,NaN,1,...,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True
35917,61151,1001101101100531211,20,NaN,NaN,1001101101100531211/2024-10-17T14:46:43.691739...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:46:44.0,NaN,1,...,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True
35918,61152,1001101101100531212,90,NaN,NaN,1001101101100531212/2024-10-17T14:46:54.523517...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:46:55.0,NaN,1,...,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True
35919,61153,1001101101100531311,20,NaN,NaN,1001101101100531311/2024-10-17T14:47:19.611355...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:47:20.0,NaN,0,...,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True


In [22]:
df_ContainerActivities = df_ContainerActivities.dropna(subset=['NEW_LAT', 'NEW_LON', 'NEW_DISTANCE'])
df_ContainerActivities.to_csv('/Users/annele/PycharmProjects/Glas-O-Mat/data/modified_data/ContainerActivities_cleaned_coordinates.csv')
df_ContainerActivities

,TRANSACTION_ID,CONTAINER_ID,SLIDER_LEVEL,CV_LEVEL,SENSOR_LEVEL,IMAGE_ID,PHONE_ID,RECORDED_AT,COMMENT,IS_EMPTIED,...,GEO_LAT,GEO_LON,DATE,CLASSIFICATION,outlier,DISTANCE,NEW_LAT,NEW_LON,NEW_DISTANCE,INACCURATE_COORDINATES
0,1,1001101101100271111,30,NaN,NaN,1001101101100271111/2023-09-28T06:36:48.022095...,TP1A.220624.014_2,2023-09-26 06:36:48.0,NaN,1,...,49.456130,7.791667,2023-09-26,accurate,False,20.382686,49.456130,7.791667,0.0,False
1,2,1001101101100271211,20,NaN,NaN,1001101101100271211/2023-09-28T06:36:05.355751...,TP1A.220624.014_2,2023-09-26 06:36:05.0,NaN,1,...,49.456130,7.791667,2023-09-26,accurate,False,20.382686,49.456130,7.791667,0.0,False
2,3,1001101101100271311,10,NaN,NaN,1001101101100271311/2023-09-28T06:37:16.029149...,TP1A.220624.014_2,2023-09-26 06:37:16.0,NaN,0,...,49.456130,7.791667,2023-09-26,accurate,False,20.382686,49.456130,7.791667,0.0,False
3,4,1001101101100191111,0,NaN,NaN,1001101101100191111/2023-09-28T06:51:15.117682...,TP1A.220624.014_2,2023-09-26 06:51:15.0,NaN,0,...,49.448608,7.769743,2023-09-26,accurate,False,8.700529,49.448608,7.769743,0.0,False
4,5,1001101101100191211,10,NaN,NaN,1001101101100191211/2023-09-28T06:51:43.824174...,TP1A.220624.014_2,2023-09-26 06:51:44.0,NaN,0,...,49.448608,7.769743,2023-09-26,accurate,False,8.700529,49.448608,7.769743,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35916,61150,1001101101100531112,90,NaN,NaN,1001101101100531112/2024-10-17T14:47:10.767846...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:47:11.0,NaN,1,...,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True
35917,61151,1001101101100531211,20,NaN,NaN,1001101101100531211/2024-10-17T14:46:43.691739...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:46:44.0,NaN,1,...,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True
35918,61152,1001101101100531212,90,NaN,NaN,1001101101100531212/2024-10-17T14:46:54.523517...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:46:55.0,NaN,1,...,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True
35919,61153,1001101101100531311,20,NaN,NaN,1001101101100531311/2024-10-17T14:47:19.611355...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:47:20.0,NaN,0,...,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True


In [23]:
df_ContainerActivities_unique = df_ContainerActivities.drop_duplicates(subset=['LOCATION_ID', 'RECORDED_AT'])
df_ContainerActivities = df_ContainerActivities_unique.copy()
df_ContainerActivities

,TRANSACTION_ID,CONTAINER_ID,SLIDER_LEVEL,CV_LEVEL,SENSOR_LEVEL,IMAGE_ID,PHONE_ID,RECORDED_AT,COMMENT,IS_EMPTIED,...,GEO_LAT,GEO_LON,DATE,CLASSIFICATION,outlier,DISTANCE,NEW_LAT,NEW_LON,NEW_DISTANCE,INACCURATE_COORDINATES
0,1,1001101101100271111,30,NaN,NaN,1001101101100271111/2023-09-28T06:36:48.022095...,TP1A.220624.014_2,2023-09-26 06:36:48.0,NaN,1,...,49.456130,7.791667,2023-09-26,accurate,False,20.382686,49.456130,7.791667,0.0,False
1,2,1001101101100271211,20,NaN,NaN,1001101101100271211/2023-09-28T06:36:05.355751...,TP1A.220624.014_2,2023-09-26 06:36:05.0,NaN,1,...,49.456130,7.791667,2023-09-26,accurate,False,20.382686,49.456130,7.791667,0.0,False
2,3,1001101101100271311,10,NaN,NaN,1001101101100271311/2023-09-28T06:37:16.029149...,TP1A.220624.014_2,2023-09-26 06:37:16.0,NaN,0,...,49.456130,7.791667,2023-09-26,accurate,False,20.382686,49.456130,7.791667,0.0,False
3,4,1001101101100191111,0,NaN,NaN,1001101101100191111/2023-09-28T06:51:15.117682...,TP1A.220624.014_2,2023-09-26 06:51:15.0,NaN,0,...,49.448608,7.769743,2023-09-26,accurate,False,8.700529,49.448608,7.769743,0.0,False
4,5,1001101101100191211,10,NaN,NaN,1001101101100191211/2023-09-28T06:51:43.824174...,TP1A.220624.014_2,2023-09-26 06:51:44.0,NaN,0,...,49.448608,7.769743,2023-09-26,accurate,False,8.700529,49.448608,7.769743,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35916,61150,1001101101100531112,90,NaN,NaN,1001101101100531112/2024-10-17T14:47:10.767846...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:47:11.0,NaN,1,...,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True
35917,61151,1001101101100531211,20,NaN,NaN,1001101101100531211/2024-10-17T14:46:43.691739...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:46:44.0,NaN,1,...,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True
35918,61152,1001101101100531212,90,NaN,NaN,1001101101100531212/2024-10-17T14:46:54.523517...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:46:55.0,NaN,1,...,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True
35919,61153,1001101101100531311,20,NaN,NaN,1001101101100531311/2024-10-17T14:47:19.611355...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:47:20.0,NaN,0,...,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True


In [24]:
# Konvertiere RECORDED_AT zu einem datetime-Objekt und extrahiere das Datum
df_ContainerActivities['DATE'] = pd.to_datetime(df_ContainerActivities['RECORDED_AT']).dt.date

# Liste der eindeutigen LOCATION_IDs
location_ids = df_ContainerActivities['LOCATION_ID'].unique()

# Funktion, um die Grafik für eine Location zu erstellen
def plot_location(location_id):
    # Filtere Daten für die ausgewählte Location
    location_data = df_ContainerActivities[df_ContainerActivities['LOCATION_ID'] == location_id]
    location_data = location_data.sort_values('DATE').reset_index()

    # Plot initialisieren
    plt.figure(figsize=(12, 6))

    # Verbindungen zwischen accurate und temporary_shift zeichnen
    for i in range(len(location_data) - 1):
        current_row = location_data.iloc[i]
        next_row = location_data.iloc[i + 1]

        if current_row['CLASSIFICATION'] in ['accurate', 'temporary_shift'] and next_row['CLASSIFICATION'] in ['accurate', 'temporary_shift']:
            # Grün: Verbindung zwischen accurate und temporary_shift
            if current_row['CLASSIFICATION'] == 'accurate' or next_row['CLASSIFICATION'] == 'accurate':
                line_color = 'green'
            else:  # Orange: Verbindung zwischen temporary_shift
                line_color = 'orange'

            plt.plot(
                [current_row['DATE'], next_row['DATE']],
                [current_row['NEW_DISTANCE'], next_row['NEW_DISTANCE']],
                color=line_color,
                linestyle='-',
                linewidth=2
            )

    # Blaue Kreuze für INACCURATE_COORDINATES
    inaccurate_coords = location_data[location_data['INACCURATE_COORDINATES'] == True]
    plt.scatter(
        inaccurate_coords['DATE'],
        inaccurate_coords['NEW_DISTANCE'],
        color='blue',
        marker='x',
        label="Inaccurate Coordinates"
    )

    # Gestrichelte Linie für DISTANCE
    plt.plot(
        location_data['DATE'],
        location_data['DISTANCE'],
        linestyle='--',
        color='purple',
        label="Original Distance"
    )

    # Rote Kreuze für Outlier
    outliers = location_data[location_data['outlier'] == True]
    plt.scatter(
        outliers['DATE'],
        outliers['DISTANCE'],
        color='red',
        marker='x',
        label="Outlier"
    )

    # Titel und Achsenbeschriftungen
    plt.title(f"Distances Over Time for Location {location_id}")
    plt.xlabel("Date")
    plt.ylabel("Distance (meters)")
    plt.xticks(rotation=45)
    plt.grid(True)

    # Legende hinzufügen
    plt.legend()

    # Layout anpassen und Plot anzeigen
    plt.tight_layout()
    plt.show()

# Interaktives Dropdown-Menü
interactive_plot = interactive(plot_location, location_id=location_ids)
display(interactive_plot)


interactive(children=(Dropdown(description='location_id', options=('100110110110027', '100110110110019', '1001…

In [25]:
df_ContainerActivities[df_ContainerActivities['INACCURATE_COORDINATES'] == True]

,TRANSACTION_ID,CONTAINER_ID,SLIDER_LEVEL,CV_LEVEL,SENSOR_LEVEL,IMAGE_ID,PHONE_ID,RECORDED_AT,COMMENT,IS_EMPTIED,...,GEO_LAT,GEO_LON,DATE,CLASSIFICATION,outlier,DISTANCE,NEW_LAT,NEW_LON,NEW_DISTANCE,INACCURATE_COORDINATES
27038,50623,1001101102100771111,70,NaN,NaN,1001101102100771111/2024-07-14T18:40:51.822636...,UPB3.230519.006_5c4ed86666236a3d,2024-07-14 18:40:52.0,NaN,1,...,49.578750,7.663861,2024-07-14,NaN,NaN,NaN,49.578750,7.663861,0.0,True
27783,51380,1001101101100691111,10,NaN,NaN,1001101101100691111/2024-07-22T06:57:51.857140...,UP1A.231005.007_feb67a335a9479a4,2024-07-22 06:57:52.0,NaN,1,...,49.447244,7.772618,2024-07-22,NaN,NaN,NaN,49.447244,7.772618,0.0,True
27784,51381,1001101101100691112,30,NaN,NaN,1001101101100691112/2024-07-22T06:58:01.494934...,UP1A.231005.007_feb67a335a9479a4,2024-07-22 06:58:01.0,NaN,1,...,49.447244,7.772618,2024-07-22,NaN,NaN,NaN,49.447244,7.772618,0.0,True
27785,51382,1001101101100691211,20,NaN,NaN,1001101101100691211/2024-07-22T06:58:09.333789...,UP1A.231005.007_feb67a335a9479a4,2024-07-22 06:58:09.0,NaN,1,...,49.447244,7.772618,2024-07-22,NaN,NaN,NaN,49.447244,7.772618,0.0,True
27786,51383,1001101101100691212,20,NaN,NaN,1001101101100691212/2024-07-22T06:58:17.138790...,UP1A.231005.007_feb67a335a9479a4,2024-07-22 06:58:17.0,NaN,1,...,49.447244,7.772618,2024-07-22,NaN,NaN,NaN,49.447244,7.772618,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35916,61150,1001101101100531112,90,NaN,NaN,1001101101100531112/2024-10-17T14:47:10.767846...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:47:11.0,NaN,1,...,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True
35917,61151,1001101101100531211,20,NaN,NaN,1001101101100531211/2024-10-17T14:46:43.691739...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:46:44.0,NaN,1,...,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True
35918,61152,1001101101100531212,90,NaN,NaN,1001101101100531212/2024-10-17T14:46:54.523517...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:46:55.0,NaN,1,...,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True
35919,61153,1001101101100531311,20,NaN,NaN,1001101101100531311/2024-10-17T14:47:19.611355...,UP1A.231005.007_0a1ae090352d8dcc,2024-10-17 14:47:20.0,NaN,0,...,49.440841,7.708089,2024-10-17,NaN,NaN,NaN,49.440841,7.708089,0.0,True


In [26]:
df_ContainerActivities[df_ContainerActivities['CLASSIFICATION'] == 'temporary_shift']

,TRANSACTION_ID,CONTAINER_ID,SLIDER_LEVEL,CV_LEVEL,SENSOR_LEVEL,IMAGE_ID,PHONE_ID,RECORDED_AT,COMMENT,IS_EMPTIED,...,GEO_LAT,GEO_LON,DATE,CLASSIFICATION,outlier,DISTANCE,NEW_LAT,NEW_LON,NEW_DISTANCE,INACCURATE_COORDINATES
222,492,1001101101100311111,10,NaN,NaN,1001101101100311111/2023-09-30T08:01:51.826430...,TP1A.220624.014_2,2023-09-28 08:01:52.0,NaN,0,...,49.453480,7.758971,2023-09-28,temporary_shift,False,60.968250,49.453948,7.759554,67.022958,False
223,493,1001101101100311211,10,NaN,NaN,1001101101100311211/2023-09-30T08:01:39.738611...,TP1A.220624.014_2,2023-09-28 08:01:40.0,NaN,0,...,49.453480,7.758971,2023-09-28,temporary_shift,False,60.968250,49.453948,7.759554,67.022958,False
224,494,1001101101100311311,30,NaN,NaN,1001101101100311311/2023-09-30T08:02:03.129807...,TP1A.220624.014_2,2023-09-28 08:02:03.0,NaN,1,...,49.453480,7.758971,2023-09-28,temporary_shift,False,60.968250,49.453948,7.759554,67.022958,False
399,766,1001101101100681111,20,NaN,NaN,1001101101100681111/2023-10-06T07:02:29.709824...,TP1A.220624.014_2,2023-10-04 07:02:30.0,NaN,1,...,49.444105,7.774833,2023-10-04,temporary_shift,False,48.585547,49.444455,7.775155,45.371893,False
400,767,1001101101100681211,20,NaN,NaN,1001101101100681211/2023-10-06T07:02:14.046707...,TP1A.220624.014_2,2023-10-04 07:02:14.0,NaN,1,...,49.444105,7.774833,2023-10-04,temporary_shift,False,48.585547,49.444455,7.775155,45.371893,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32062,56385,1001101102100991211,10,NaN,NaN,1001101102100991211/2024-08-29T10:08:55.868999...,TP1A.220624.014_4ac8e8be5475e6a6,2024-08-29 10:08:56.0,NaN,0,...,49.461889,7.555111,2024-08-29,temporary_shift,False,44.447373,49.462131,7.555849,59.927348,False
32063,56386,1001101102100991311,40,NaN,NaN,1001101102100991311/2024-08-29T10:08:35.303077...,TP1A.220624.014_4ac8e8be5475e6a6,2024-08-29 10:08:35.0,NaN,1,...,49.461889,7.555111,2024-08-29,temporary_shift,False,44.447373,49.462131,7.555849,59.927348,False
32136,56459,1001101102101021111,10,NaN,NaN,1001101102101021111/2024-08-29T13:58:10.691828...,TP1A.220624.014_4ac8e8be5475e6a6,2024-08-29 13:58:11.0,NaN,1,...,49.477833,7.652722,2024-08-29,temporary_shift,False,43.400220,49.478208,7.652797,42.017673,False
32137,56460,1001101102101021211,10,NaN,NaN,1001101102101021211/2024-08-29T13:58:03.016177...,TP1A.220624.014_4ac8e8be5475e6a6,2024-08-29 13:58:03.0,NaN,0,...,49.477833,7.652722,2024-08-29,temporary_shift,False,43.400220,49.478208,7.652797,42.017673,False
